In [ ]:
from IPython.core.display import HTML
with open('../style.css', 'r') as file:
    css = file.read()
HTML(css)

In [ ]:
%load_ext nb_mypy

In [ ]:
Variable = str
Formula  = str

# The Knight's Tour

This notebook computes a solution to the [knight's tour](https://en.wikipedia.org/wiki/Knight%27s_tour) using the constraint solver `Z3`.  Furthrmore, we will compute a closed tour, i.e. the knight will return to its starting square in its last move.

In [ ]:
import z3

Given an integer from the set $\{0, 1, \cdots, 64\}$, the function `row(i)` computes the name of the variable that specifies the *row* of the knight after its $i^{\textrm{th}}$ move.

In [ ]:
def row(i: int) -> Variable:
    return f'R{i}'

Given an integer from the set $\{0, 1, \cdots, 64\}$, the function `row(i)` computes the name of the variable that specifies the *column* of the knight after its $i^{\textrm{th}}$ move.

In [ ]:
def col(i: int) -> Variable:
    return f'C{i}'

The function `all_variables` computes the names of all variables.

In [ ]:
def all_variables() -> set[Variable]:
    Variables = set()
    for i in range(64+1):
        Variables.add(row(i))
        Variables.add(col(i))
    return Variables

The function `is_knight_move(i)` returns a formula that specifies that the $i^{\textrm{th}}$ move is a legal move for a knight.
In order to form the *conjunction* of two formulas we use the function `z3.And`, while the *disjunction* is build with the function `z3.Or`.

The figure below shows the moves of a knight:  The knight on `e4` can jump to all red squares.
<img src="knight-moves.png" width="50%">

In [ ]:
def is_knight_move(i: int) -> Formula:
    r  = row(i)
    c  = col(i)
    rX = row(i+1)
    cX = col(i+1)
    Formulas = set()
    for delta_r, delta_c in [(1, 2), (2, 1)]:
        Formulas.add(f'z3.And({rX} == {r} + {delta_r}, {cX} == {c} + {delta_c})')
        Formulas.add(f'z3.And({rX} == {r} + {delta_r}, {cX} + {delta_c} == {c})')
        Formulas.add(f'z3.And({rX} + {delta_r} == {r}, {cX} == {c} + {delta_c})')
        Formulas.add(f'z3.And({rX} + {delta_r} == {r}, {cX} + {delta_c} == {c})') 
    return 'z3.Or(' + ', '.join(Formulas) + ')'

In [ ]:
is_knight_move(0)

The function `all_different` computes a set of formulas specifiying that
$$ \langle \textrm{R}i, \textrm{C}i \rangle \not= \langle \textrm{R}j, \textrm{C}j \rangle $$
provided that $i \not= j$.  This specifies that, with the exception of the starting square, the knight will not not visit a square twice.  This way, we ensure that all squares are visited as the tour has a length of 64 and there are exactly $8 \times 8 = 64$ squares on a chessboard.

In [ ]:
def all_different() -> set[Formula]:
    Result = set()
    for i in range(62+1):
        for j in range (i+1, 63+1):
            Result.add(f'z3.Or({row(i)} != {row(j)}, {col(i)} != {col(j)})')
    return Result

In [ ]:
def all_constraints() -> set[Formula]:
    Constraints = all_different()
    Constraints.add(f'{row(0 )} == 0')
    Constraints.add(f'{col(0 )} == 0')
    Constraints.add(f'{row(64)} == 0')
    Constraints.add(f'{col(64)} == 0')
    for i in range(63+1):
        Constraints.add(is_knight_move(i))
    for i in range(64+1):
        Constraints.add(f'{row(i)} >= 0')
        Constraints.add(f'{col(i)} >= 0')
    return Constraints

In [ ]:
len(all_constraints())

The function `solve(Constraints, Variables)` receives two arguments:
- `Constraints` is a set of formulas representing a constraint satisfaction problem.
- `Variables`   is the set of variables that occur in this formulas.

   It is assumed that all variables can be presented by bit-vector of length 4.
   We need 4 bits because the numbers use a sign bit.

The function computes a solution to the given problem and returns this solution.

In [ ]:
def solve(Constraints: set[Formula], Variables: set[Variable]) -> dict[Variable, z3.z3.BitVecNumRef] | None:
    Environment: dict = {}
    exec('import z3', Environment)
    for v in Variables:
        exec(f'{v} = z3.BitVec(f"{v}", 4)', Environment)
    S = z3.Solver()
    for c in Constraints:
        S.add(eval(c, Environment))
    result = str(S.check())
    if result == 'sat':
        Model    = S.model()
        Solution = { v: Model[eval(v, Environment)] for v in Variables }
        return Solution
    elif result == 'unsat':
        print('The problem is not solvable.')
    else:
        print('Z3 cannot determine whether the problem is solvable.')
    return None

It takes about 13 minutes to compute the solution.

In [ ]:
%%time
Solution = solve(all_constraints(), all_variables())
Solution

The type declaration below is needed because the `%%time` magic does not play along with the type checker.

In [ ]:
Solution: dict[Variable, z3.z3.BitVecNumRef]

The function `create_board(Solution)` returns a matrix `Board` of size $8\times 8$.
The following holds:
$$ \texttt{Board}[\texttt{R}i][\texttt{C}i] = i $$
Therefore, if `Board[r][c] == i`, then at the beginning of the $i^{\textrm{th}}$ move the knight is located in row `r` and column `c`. 

In [ ]:
def create_board(Solution: dict[Variable, z3.z3.BitVecNumRef]) -> list[list[int]]:
    Board = [[0 for _ in range(8)] for _ in range(8)]
    for i in range(1, 65):
        r = Solution[row(i)].as_long()
        c = Solution[col(i)].as_long()
        Board[r][c] = i
    return Board

In [ ]:
create_board(Solution)

The function `print_board` prints the given `Board`.

In [ ]:
def print_board(Board: list[list[int]]) -> None:
    n = len(Board)
    # Determine the width of the widest element in the matrix
    width = max([ len(str(element)) for row in Board
                                    for element in row
                ])
    # Create the top and bottom of the matrix
    top_line = '╔'
    for i in range(n - 1):
        top_line += '═' * (width + 2) + '╦'
    top_line += '═' * (width + 2) + '╗'
    mid_line = '╠'
    for i in range(n - 1):
        mid_line += '═' * (width + 2) + '╬'
    mid_line += '═' * (width + 2) + '╣'    
    bot_line = '╚'
    for i in range(n - 1):
        bot_line += '═' * (width + 2) + '╩'
    bot_line += '═' * (width + 2) + '╝'
    # Print the top of the matrix
    print(top_line)
    # Iterate through the rows and columns of the matrix, and print
    # each element with proper padding
    for i, row in enumerate(Board):
        line = '\u2551'
        for element in row:
            line += f' {element:>{width}} ║'
        print(line)
        # Print a horizontal line
        if i < len(Board) - 1:
            print(mid_line)
    # Print the bottom of the matrix
    print(bot_line)

In [ ]:
print_board(create_board(Solution))

# Visualization

If you have not yet installed `chess-problem-visuals` you have to uncomment the following line.

In [ ]:
!pip install git+https://github.com/reclinarka/chess-problem-visuals

In [ ]:
from chess_problem_visuals import problem_board

The function `show_solution` displays the given solution on a chessboard.
The solution `Board` is represented as a list of lists.  We have `Board[row][col] == k` if the $k^\textrm{th}$ move leads the knight to the position `(row, col)`.

In [ ]:
def show_solution(Board, width="50%"):
    n         = len(Board)
    Positions = {}
    for row in range(n):
        for col in range(n):
            k = Board[row][col]
            Positions[k] = (row, col)
    start = (0, 0)
    Path  = [start]
    for k in range(1, n*n+1):
        Path.append(Positions[k])
    Visual = problem_board(n, K_start=start, K_path=Path, 
                           html_width=width, 
                           arrow_color="darkgreen",
                           arrow_width=0.2)
    return Visual

In [ ]:
show_solution(create_board(Solution))